This notebook runs from the Anaconda distribution and has the "findspark" Python Module installed.
Refer to https://www.learntospark.com/2019/12/install-spark-in-windows-using-jupyter-notebook.html

In [1]:
import findspark
findspark.init()
findspark.find()

'/usr/local/opt/apache-spark/libexec'

In [3]:
from pyspark.sql import SparkSession

from pyspark.sql.functions import *

In [4]:
# Create a SparkSession
spark = (SparkSession
    .builder
    .appName("SparkFlightDelaysApp")
    .getOrCreate())

# file to read
delay_csv_file = "data/departuredelays.csv"

22/04/02 23:10:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/04/02 23:10:09 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [6]:
# Read and create a temporary view
# infer schema (may want to specify 
# for larger files)
df = (spark.read.format("csv")
    .option("samplingRatio", 0.001)
    .option("inferSchema", "true")
    .option("header", "true")
    .load(delay_csv_file))
df.createOrReplaceTempView("us_delay_flights_tbl")

In [7]:
# find flights with a distance of
# > 1000 miles
spark.sql("""SELECT distance, origin, destination
    FROM us_delay_flights_tbl WHERE distance > 1000 
    ORDER BY distance DESC""").show(10)

+--------+------+-----------+
|distance|origin|destination|
+--------+------+-----------+
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
+--------+------+-----------+
only showing top 10 rows



In [8]:
# flights between San Francisco (SFO) and 
# Chicago (ORD) with at least a 2 hour delay
spark.sql("""SELECT date, delay, origin, destination 
    FROM us_delay_flights_tbl 
    WHERE delay > 120 AND ORIGIN = 'SFO' AND DESTINATION = 'ORD' 
    ORDER BY delay DESC""").show(10)

+-------+-----+------+-----------+
|   date|delay|origin|destination|
+-------+-----+------+-----------+
|2190925| 1638|   SFO|        ORD|
|1031755|  396|   SFO|        ORD|
|1022330|  326|   SFO|        ORD|
|1051205|  320|   SFO|        ORD|
|1190925|  297|   SFO|        ORD|
|2171115|  296|   SFO|        ORD|
|1071040|  279|   SFO|        ORD|
|1051550|  274|   SFO|        ORD|
|3120730|  266|   SFO|        ORD|
|1261104|  258|   SFO|        ORD|
+-------+-----+------+-----------+
only showing top 10 rows



In [9]:
# label flights with an indication of delay experienced
spark.sql("""SELECT delay, origin, destination,
                CASE
                    WHEN delay > 360 THEN 'Very Long Delays'
                    WHEN delay > 120 AND delay < 350 THEN 'Long Delays'
                    WHEN delay > 60 AND delay < 120 THEN 'Short Delays'
                    WHEN delay > 0 AND delay < 50 THEN 'Tolerable Delays'
                    WHEN delay = 0 THEN 'No Delays'
                    ELSE 'Early'
                END AS Flight_Delays
                FROM us_delay_flights_tbl
                ORDER BY origin, delay DESC""").show(10)

+-----+------+-----------+-------------+
|delay|origin|destination|Flight_Delays|
+-----+------+-----------+-------------+
|  333|   ABE|        ATL|  Long Delays|
|  305|   ABE|        ATL|  Long Delays|
|  275|   ABE|        ATL|  Long Delays|
|  257|   ABE|        ATL|  Long Delays|
|  247|   ABE|        DTW|  Long Delays|
|  247|   ABE|        ATL|  Long Delays|
|  219|   ABE|        ORD|  Long Delays|
|  211|   ABE|        ATL|  Long Delays|
|  197|   ABE|        DTW|  Long Delays|
|  192|   ABE|        ORD|  Long Delays|
+-----+------+-----------+-------------+
only showing top 10 rows

